# Create Raster Stack of SMAP data

## Objectives

- Read SMAP data from h5 file
- Create raster
- Add raster to raster stack, and save as geotiff file

## Prerequisites

Necessary libraries- 
    - rhdf5
    - raster
    - rdgal
    - sp
Install these packages using the following code

In [ ]:
install.packages("sp",repos="http://cran.rstudio.com/")
install.packages("raster",repos="http://cran.rstudio.com/")
source("https://bioconductor.org/biocLite.R")
biocLite("rgdal")
biocLite("rhdf5")

If you already have these packages installed confirm that you are using the most up-to-date versions

In [ ]:
update.packages("sp",repos="http://cran.rstudio.com/")
update.packages("raster",repos="http://cran.rstudio.com/")
source("https://bioconductor.org/biocLite.R")
biocLite("rgdal")
biocLite("rhdf5")

## Getting Data

To access the SMAP data call download.file, with the first argument being the url where the data is located, and the second being the file name and path where you want to store the data this can be repeated with as many files as are needed

In [ ]:
download.file('ftp://n5eil01u.ecs.nsidc.org/SAN/SMAP/SPL4SMGP.001/2015.04.01/SMAP_L4_SM_gph_20150401T013000_Vb1010_001.h5',
             'SMAP_1.h5')

## Reading SMAP

Include necessary packages, and find the dataset(s) you would like to read in

In [ ]:
library(sp)
library(rhdf5)
library(raster)
library(rgdal)

#File name and group where data is located
input_file=paste0('SMAP.h5')#file name and path
group='Geophysical_Data'
h5f=H5Fopen(input_file)
h5f&'Geophysical_Data'#Print out all datasets in the geophysical data group

Read data from each desired dataset from the h5 file, and create raster layer to be added to the stack

In [ ]:
#Define datasets
dataset=c('sm_rootzone','heat_flux_ground')#possible sets to read in
numSets=length(dataset)
x=vector("list",numSets)#list containing all raster objects
missing_vals=-9999
#For loop to read in data from each set until 
for (i in 1:numSets){
    #read h5 file and get data from desired set
    input <- h5read(input_file, paste0('/', group, '/', dataset[[i]]))
    #exclude missing values
    input[input == missing_vals] <- NA
    #create raster and transpose
    r <- raster(t(input))
    #create vector of lat/lon values
    lat <- apply(h5read(input_file, '/cell_lat'), 2, unique)
    lon <- apply(h5read(input_file, '/cell_lon'), 2, unique)
    #geographical crs input
    crs = '+proj=longlat +lat_0=0 +lon_0=0 +lat_ts=30 
                    +ellps=WGS84 +datum=WGS84 +units=m'
    crs_in = '+proj=cea +lat_0=0 +lon_0=0 +lat_ts=30 
                    +ellps=WGS84 +datum=WGS84 +units=m'
    #create raster using lon/lat and project raster onto crs_in to find extent of data
    ex <- extent(projectExtent(raster(extent(range(lon), range(lat)),
                                        crs),
                                        crs_in))
    extent(r) <- ex
    projection(r) <- CRS(crs_in)
    crs_out = "+init=epsg:4326"#defines global coordinate reference system
    smap <- projectRaster(r, crs = crs_out)#create final raster projected on global crs
    x[[i]]<-smap#add raster to list
}#end for loop

## Creating Raster Stack

The stack function takes a list of raster ojects as an argument and creates a raster stack with each layer being one of the rasters in the list

In [ ]:
#create raster stack
stck <-stack(x)
#define layer names 
for(i in 1:numSets){
    names(stck)[i]<-datasets[[i]]
}
#save raster stack to a geotiff file
writeRaster(stck,'smap_stack.tif', format = 'GTiff', overwrite=TRUE)